In [344]:
import pandas as pd
import numpy as np

In [345]:
BoM = ['ABC:2', 'XYZ:1', 'IJK:1', 'ABC:1', 'IJK:1', 'XYZ:2', 'DEF:2']
Disti = ['XYZ:2', 'GEF:2', 'ABC:4', 'IJK:2']
L = lambda s: [s.split(':')[0], int(s.split(':')[1])]

BoM= [L(s)for s in BoM]
Disti = [L(s) for s in Disti]

df = pd.DataFrame(BoM,columns=['Part Number','Quantity'])
df1 = pd.DataFrame(Disti,columns=['Part Number','Quantity'])
print(df)
print(df1)

  Part Number  Quantity
0         ABC         2
1         XYZ         1
2         IJK         1
3         ABC         1
4         IJK         1
5         XYZ         2
6         DEF         2
  Part Number  Quantity
0         XYZ         2
1         GEF         2
2         ABC         4
3         IJK         2


### Get Sequence (order) for df1

In [346]:
s = df['Part Number'].drop_duplicates()
s = s[s.isin(df1['Part Number'])]
s

0    ABC
1    XYZ
2    IJK
Name: Part Number, dtype: object

In [347]:
s1= df1[~df1['Part Number'].isin(s)]['Part Number']
s1

1    GEF
Name: Part Number, dtype: object

In [348]:
keeporder = pd.concat([s, s1])
keeporder

0    ABC
1    XYZ
2    IJK
1    GEF
Name: Part Number, dtype: object

In [349]:
df1 = df1.iloc[list(map(df1['Part Number'].tolist().index, keeporder))].set_index('Part Number').reset_index()
df1

,Part Number,Quantity
0,ABC,4
1,XYZ,2
2,IJK,2
3,GEF,2


### Get df1 target with respect to df

In [350]:
df1TargetWrt_df = df[['Part Number','Quantity']].merge(df1[['Part Number']],how='right')
df1TargetWrt_df['Quantity'] = df1TargetWrt_df['Quantity'].fillna(0).astype('Int64')
df1TargetWrt_df

,Part Number,Quantity
0,ABC,2
1,ABC,1
2,XYZ,1
3,XYZ,2
4,IJK,1
5,IJK,1
6,GEF,0


In [351]:
cols = ['Part Number','Quantity']
quantities = df1TargetWrt_df[cols].groupby('Part Number') \
                 .apply(lambda x: x.set_index(['Part Number']).to_dict('list')) \
                 .to_dict()
key_order = df1TargetWrt_df['Part Number'].drop_duplicates()
targetPNQuantities = {}
for k in key_order:
    targetPNQuantities[k] = quantities[k]
# requiredQuantities = list(targetPNQuantities.values())#[3]['Quantity']
# print(requiredQuantities)
targetPNQuantities

{'ABC': {'Quantity': [2, 1]},
 'XYZ': {'Quantity': [1, 2]},
 'IJK': {'Quantity': [1, 1]},
 'GEF': {'Quantity': [0]}}

### Get indexes for df1 quantities of each PartNumber to add using groupby on those Indexes

In [352]:
def make_groups(Quantities, getPartNumber, targetDict):
    targets = targetDict[getPartNumber]['Quantity']

    if not targets[0]:
        result = np.zeros(len(Quantities), dtype = int)
        return result
    
    result = np.empty(len(Quantities),dtype=np.uint64)

    total = 0
    group = 0

    for i,x_i in enumerate(Quantities):
        total += x_i
        if total > targets[0]:
            targets= targets[1:]
            group += 1
            total = 1
        
        result[i] = group
    
    return result.tolist()

In [353]:
df1 = df1.reindex(df1.index.repeat(df1['Quantity'])).assign(Quantity=1)
df1

,Part Number,Quantity
0,ABC,1
0,ABC,1
0,ABC,1
0,ABC,1
1,XYZ,1
1,XYZ,1
2,IJK,1
2,IJK,1
3,GEF,1
3,GEF,1


In [354]:
g = df1.groupby("Part Number")["Part Number", "Quantity"].apply( lambda x: make_groups(pd.DataFrame(x)["Quantity"].to_numpy(), pd.DataFrame(x)["Part Number"].to_numpy()[0], targetPNQuantities))
g

C:\Users\kalee\AppData\Local\Temp\ipykernel_9832\1508180741.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  g = df1.groupby("Part Number")["Part Number", "Quantity"].apply( lambda x: make_groups(pd.DataFrame(x)["Quantity"].to_numpy(), pd.DataFrame(x)["Part Number"].to_numpy()[0], targetPNQuantities))


Part Number
ABC    [0, 0, 1, 2]
GEF          [0, 0]
IJK          [0, 1]
XYZ          [0, 1]
dtype: object

In [355]:
h =pd.DataFrame(g).reset_index()
h.columns = df.columns[:1].tolist() +['Quantity']
h = h.explode('Quantity')
index = h.index
h = pd.merge(keeporder, h,
         how='left', on='Part Number',
         sort=False
         ).set_index(index)
h

,Part Number,Quantity
0,ABC,0
0,ABC,0
0,ABC,1
0,ABC,2
1,XYZ,0
1,XYZ,1
2,IJK,0
2,IJK,1
3,GEF,0
3,GEF,0


In [356]:
h['Quantity']

0    0
0    0
0    1
0    2
1    0
1    1
2    0
2    1
3    0
3    0
Name: Quantity, dtype: object

In [357]:
(df1.groupby(by=["Part Number", h['Quantity']])
        .sum())

Quantity
Part Number Quantity          
ABC         0                2
            1                1
            2                1
GEF         0                2
IJK         0                1
            1                1
XYZ         0                1
            1                1

In [358]:
df1targetresults= (df1.groupby(by=["Part Number", h['Quantity']])
        .sum()
        .reset_index(1, drop=True)
        .reset_index())
df1targetresults

,Part Number,Quantity
0,ABC,2
1,ABC,1
2,ABC,1
3,GEF,2
4,IJK,1
5,IJK,1
6,XYZ,1
7,XYZ,1


### Sort those df1target and df

In [359]:
df1target_sort = df1targetresults.rename_axis('MyIdx').sort_values(by = ['Part Number', 'MyIdx'], ascending=[True, True])
df1target_sort

,Part Number,Quantity
MyIdx,,
0,ABC,2
1,ABC,1
2,ABC,1
3,GEF,2
4,IJK,1
5,IJK,1
6,XYZ,1
7,XYZ,1


In [360]:
df

,Part Number,Quantity
0,ABC,2
1,XYZ,1
2,IJK,1
3,ABC,1
4,IJK,1
5,XYZ,2
6,DEF,2


In [361]:
dfsort= df.rename_axis('MyIdx').sort_values(by = ['Part Number', 'MyIdx'], ascending=[True, True])
dfsort

,Part Number,Quantity
MyIdx,,
0,ABC,2
3,ABC,1
6,DEF,2
2,IJK,1
4,IJK,1
1,XYZ,1
5,XYZ,2


### Get Matching indexes from df1 (df1target_sort) when compared with df (dfsort)

In [362]:
def getMatchingIndexfrom(df1, df):
    compareColumn = []
    compareColumnIndex =[]
    checked = [-1]
    for index, row in df.iterrows():
        df1Row = df1[df1["Part Number"] == row["Part Number"]]
        if df1Row.shape[0] == 0:
            compareColumn.append("Part Number and Quantity not available")
            compareColumnIndex.append(-1)
        else:
            check = False
            for index1, row1 in df1Row.iterrows():
                if index1 not in checked and row1["Quantity"] == row["Quantity"]:
                    checked.append(index1)
                    compareColumn.append(f"Both matching at index {index1}")
                    compareColumnIndex.append(index1)
                    check = True
                    break
            if check == False:
                compareColumn.append(f"Quantity not matching at index {index1}")
                compareColumnIndex.append(index1)
    df["compare"] = compareColumn
    df["compareIndex"] = compareColumnIndex
    df.sort_values(by=['MyIdx'], inplace= True)
    return df

In [363]:
df = getMatchingIndexfrom(df1target_sort,dfsort)
df

,Part Number,Quantity,compare,compareIndex
MyIdx,,,,
0,ABC,2,Both matching at index 0,0
1,XYZ,1,Both matching at index 6,6
2,IJK,1,Both matching at index 4,4
3,ABC,1,Both matching at index 1,1
4,IJK,1,Both matching at index 5,5
5,XYZ,2,Quantity not matching at index 7,7
6,DEF,2,Part Number and Quantity not available,-1


### Merging df with df1target_sort by indexes of df1target_sort to get Final Target

In [364]:
final = df.drop('compare', axis=1).merge(df1target_sort, left_on='compareIndex', right_on='MyIdx', how= 'outer').drop('compareIndex', axis=1)
final["Error Flag"]= np.where(final['Quantity_x']==final['Quantity_y'], "", "x")
final['Quantity_x'] = final['Quantity_x'].fillna(-1).astype('Int64').astype(str)
final['Quantity_y'] = final['Quantity_y'].fillna(-1).astype('Int64').astype(str)
final=final.fillna('').replace('-1', ' ')
final

,Part Number_x,Quantity_x,Part Number_y,Quantity_y,Error Flag
0,ABC,2,ABC,2,
1,XYZ,1,XYZ,1,
2,IJK,1,IJK,1,
3,ABC,1,ABC,1,
4,IJK,1,IJK,1,
5,XYZ,2,XYZ,1,x
6,DEF,2,,,x
7,,,ABC,1,x
8,,,GEF,2,x


In [366]:
final.columns = ["BoM PN", "BoM Qty", "Disti PN", "Disti Qty", "Error Flag"]
final

,BoM PN,BoM Qty,Disti PN,Disti Qty,Error Flag
0,ABC,2,ABC,2,
1,XYZ,1,XYZ,1,
2,IJK,1,IJK,1,
3,ABC,1,ABC,1,
4,IJK,1,IJK,1,
5,XYZ,2,XYZ,1,x
6,DEF,2,,,x
7,,,ABC,1,x
8,,,GEF,2,x


In [368]:
unifiedDict =final.to_dict()
unifiedDict

{'BoM PN': {0: 'ABC',
  1: 'XYZ',
  2: 'IJK',
  3: 'ABC',
  4: 'IJK',
  5: 'XYZ',
  6: 'DEF',
  7: '',
  8: ''},
 'BoM Qty': {0: '2',
  1: '1',
  2: '1',
  3: '1',
  4: '1',
  5: '2',
  6: '2',
  7: ' ',
  8: ' '},
 'Disti PN': {0: 'ABC',
  1: 'XYZ',
  2: 'IJK',
  3: 'ABC',
  4: 'IJK',
  5: 'XYZ',
  6: '',
  7: 'ABC',
  8: 'GEF'},
 'Disti Qty': {0: '2',
  1: '1',
  2: '1',
  3: '1',
  4: '1',
  5: '1',
  6: ' ',
  7: '1',
  8: '2'},
 'Error Flag': {0: '',
  1: '',
  2: '',
  3: '',
  4: '',
  5: 'x',
  6: 'x',
  7: 'x',
  8: 'x'}}